## 模型结构

- 两层单向LSTM, 输出序列结果, 即(batch_size, step_size, feature_size)
- 分别输入到1, 2, 3, 4, 5, 6共6个不同长度的卷积层中
- 卷积层为单层
- 对于每个问题, 将所有卷积核结果并起来
- 将两个问题并起来的结果, 分别[相减并取绝对值], [相乘], 再将所有的结果合并
- 输出到两层Dense层中, 最后一层为sigmoid激活函数, 输出结果

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from glob import glob
from datetime import datetime

In [2]:
DATA_PATH = "./data/"
TRAIN_PATH = DATA_PATH + "train.csv"
TEST_PATH = DATA_PATH + "test.csv"
WORD_EMBED_PATH = DATA_PATH + "word_embed.txt"
CHAR_EMBED_PATH = DATA_PATH + "char_embed.txt"
QUEST_PATH = DATA_PATH + "question.csv"

train_data = pd.read_csv(TRAIN_PATH)
test_data = pd.read_csv(TEST_PATH)
question_data = pd.read_csv(QUEST_PATH)
word_embedding_data = pd.read_csv(WORD_EMBED_PATH, delimiter=" ", header=None, index_col=0)
char_embedding_data = pd.read_csv(CHAR_EMBED_PATH, delimiter=" ", header=None, index_col=0)

question_data["words"] = question_data["words"].str.split(" ")
question_data["chars"] = question_data["chars"].str.split(" ")

label = train_data["label"].values

from keras.preprocessing.text import Tokenizer

MAX_COUNT = 10000

word_tokenizer = Tokenizer(MAX_COUNT)
word_tokenizer.fit_on_texts(question_data["words"])

word_embedding_data = np.concatenate(
    (
        np.zeros(shape=(1, word_embedding_data.shape[1]), dtype=np.float64),
        word_embedding_data.loc[list(word_tokenizer.word_index.keys())[:MAX_COUNT]].values
    ),
    axis=0
)

char_tokenizer = Tokenizer(MAX_COUNT)
char_tokenizer.fit_on_texts(question_data["chars"])

char_embedding_data = np.concatenate(
    (
        np.zeros(shape=(1, char_embedding_data.shape[1]), dtype=np.float64),
        char_embedding_data.loc[list(char_tokenizer.word_index.keys())[:MAX_COUNT]].values
    ),
    axis=0
)

word_embedding_data.shape, char_embedding_data.shape

Using TensorFlow backend.


((10001, 300), (3049, 300))

In [3]:
from keras.preprocessing.sequence import pad_sequences

SEQ_LEN = 25

def gen_word_data(data):
    seq_word1 = word_tokenizer.texts_to_sequences(data.merge(question_data, how="left", left_on="q1", right_on="qid")["words"])
    seq_word2 = word_tokenizer.texts_to_sequences(data.merge(question_data, how="left", left_on="q2", right_on="qid")["words"])
    return pad_sequences(seq_word1, maxlen=SEQ_LEN, padding="pre",truncating="pre"), \
        pad_sequences(seq_word2, maxlen=SEQ_LEN, padding="pre",truncating="pre")
    
def gen_char_data(data):
    seq_char1 = char_tokenizer.texts_to_sequences(data.merge(question_data, how="left", left_on="q1", right_on="qid")["chars"])
    seq_char2 = char_tokenizer.texts_to_sequences(data.merge(question_data, how="left", left_on="q2", right_on="qid")["chars"])
    return pad_sequences(seq_char1, maxlen=SEQ_LEN, padding="pre",truncating="pre"), \
        pad_sequences(seq_char2, maxlen=SEQ_LEN, padding="pre",truncating="pre")

word1, word2 = gen_word_data(train_data)
char1, char2 = gen_char_data(train_data)
test_word1, test_word2 = gen_word_data(test_data)
test_char1, test_char2 = gen_char_data(test_data)

word1.shape, word2.shape, test_word1.shape, test_word2.shape, char1.shape, char2.shape, test_char1.shape, test_char2.shape

((254386, 30),
 (254386, 30),
 (172956, 30),
 (172956, 30),
 (254386, 30),
 (254386, 30),
 (172956, 30),
 (172956, 30))

In [4]:
from keras.models import Model
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM, Bidirectional, TimeDistributed
from keras.layers import Conv1D, MaxPool1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dropout, BatchNormalization, Dense, Flatten, Lambda, K

In [5]:
# general
NUM_EPOCHES = 30
EPOCHES1 = 3
EPOCHES2 = 5
EPOCHES3 = 22
BATCH_SIZE = 1024
DROP_RATE = 0.3
PATIENCE = 6

# cnn
CONV_LEN1 = 128
CONV_LEN2 = 128
CONV_LEN3 = 128
CONV_LEN4 = 128
CONV_LEN5 = 128
CONV_LEN6 = 128
CONV_LEN = CONV_LEN1 + CONV_LEN2 + CONV_LEN3 + CONV_LEN4 + CONV_LEN5 + CONV_LEN6

# lstm
LSTM_SIZE1 = 256
LSTM_SIZE2 = 256
LSTM_DROP_RATE = 0.3

# dense
DENSE_SIZE1 = 512
DENSE_SIZE2 = 256

In [6]:
def cnn_layer1(inputa, inputb, filters, kernel_size):
    conv = Conv1D(filters=filters, kernel_size=kernel_size, padding="same", activation="relu")
    conv_outputa = conv(inputa)
    conv_outputa = GlobalAveragePooling1D()(conv_outputa)
    conv_outputb = conv(inputb)
    conv_outputb = GlobalAveragePooling1D()(conv_outputb)
    return conv_outputa, conv_outputb
    
def cnn_layer2(inputa, inputb, filters, kernel_size):
    conv = Conv1D(filters=filters, kernel_size=kernel_size, padding="same", activation="relu")
    conv_outputa = conv(inputa)
    conv_outputa = MaxPool1D(pool_size=SEQ_LEN)(conv_outputa)
    conv_outputa = Flatten()(conv_outputa)
    conv_outputb = conv(inputb)
    conv_outputb = MaxPool1D(pool_size=SEQ_LEN)(conv_outputb)
    conv_outputb = Flatten()(conv_outputb)
    return conv_outputa, conv_outputb

def cnn_layer3(inputa, inputb, filters, kernel_size):
    conv = Conv1D(filters=filters, kernel_size=kernel_size, padding="same", activation="relu")
    
    conv_outputa = conv(inputa)
    conv_outputa1 = Flatten()(MaxPool1D(pool_size=SEQ_LEN)(conv_outputa))
    conv_outputa2 = GlobalAveragePooling1D()(conv_outputa)
    conv_outputa = concatenate([conv_outputa1, conv_outputa2])
    
    conv_outputb = conv(inputb)
    conv_outputb1 = Flatten()(MaxPool1D(pool_size=SEQ_LEN)(conv_outputb))
    conv_outputb2 = GlobalAveragePooling1D()(conv_outputb)
    conv_outputb = concatenate([conv_outputb1, conv_outputb2])
    
    return conv_outputa, conv_outputb

# WORDS

In [7]:
from sklearn.model_selection import StratifiedKFold

best_results = []
last_results = []

for i, (train_index, dev_index) in enumerate(StratifiedKFold(n_splits=10).split(X=word1, y=label)):  # word/char switch
    train_x1, train_x2, train_y = word1[train_index, :], word2[train_index, :], label[train_index]  # word/char switch
    dev_x1, dev_x2, dev_y = word1[dev_index, :], word2[dev_index, :], label[dev_index]  # word/char switch
    
    input1 = Input(shape=(SEQ_LEN,), dtype="int32")
    input2 = Input(shape=(SEQ_LEN,), dtype="int32")

    embedding_layer = Embedding(
        input_dim=word_embedding_data.shape[0],  # word/char switch
        output_dim=word_embedding_data.shape[1],  # word/char switch
        weights=[word_embedding_data],  # word/char switch
        input_length=SEQ_LEN,
        trainable=False
    )
    
    vector1 = embedding_layer(input1)
    vector2 = embedding_layer(input2)
    
    lstm_layer1 = LSTM(LSTM_SIZE1, dropout=LSTM_DROP_RATE, recurrent_dropout=LSTM_DROP_RATE, return_sequences=True)
    layer1a = lstm_layer1(vector1)
    layer1a = Dropout(LSTM_DROP_RATE)(layer1a)
    layer1b = lstm_layer1(vector2)
    layer1b = Dropout(LSTM_DROP_RATE)(layer1b)

    lstm_layer2 = LSTM(LSTM_SIZE2, dropout=LSTM_DROP_RATE, recurrent_dropout=LSTM_DROP_RATE, return_sequences=True)
    layer2a = lstm_layer2(layer1a)
    layer2b = lstm_layer2(layer1b)
#     # 每个序列片拼接对应的原始embedding向量
#     layer2a = concatenate([vector1, layer2a])
#     layer2b = concatenate([vector2, layer2b])
    
    conv1a, conv1b = cnn_layer3(layer2a, layer2b, filters=CONV_LEN1, kernel_size=1)
    conv2a, conv2b = cnn_layer3(layer2a, layer2b, filters=CONV_LEN2, kernel_size=2)
    conv3a, conv3b = cnn_layer3(layer2a, layer2b, filters=CONV_LEN3, kernel_size=3)
    conv4a, conv4b = cnn_layer3(layer2a, layer2b, filters=CONV_LEN4, kernel_size=4)
    conv5a, conv5b = cnn_layer3(layer2a, layer2b, filters=CONV_LEN5, kernel_size=5)
    conv6a, conv6b = cnn_layer3(layer2a, layer2b, filters=CONV_LEN6, kernel_size=6)
    
    merge_a = concatenate([conv1a, conv2a, conv3a, conv4a, conv5a, conv6a])
    merge_b = concatenate([conv1b, conv2b, conv3b, conv4b, conv5b, conv6b])
    diff = Lambda(lambda x: K.abs(x[0] - x[1]))([merge_a, merge_b])
    mult = Lambda(lambda x: x[0] * x[1])([merge_a, merge_b])
    merge = concatenate([diff, mult])
    
    x = Dropout(DROP_RATE)(merge)
    x = BatchNormalization()(x)
    x = Dense(DENSE_SIZE1, activation="relu")(x)
    x = Dropout(DROP_RATE)(x)
    x = BatchNormalization()(x)
    x = Dense(DENSE_SIZE2, activation="relu")(x)
    x = Dropout(DROP_RATE)(x)
    x = BatchNormalization()(x)
    pred = Dense(1, activation="sigmoid")(x)
    
    model = Model(inputs=[input1, input2], outputs=pred)
    model.compile(
        optimizer="nadam",
        loss="binary_crossentropy",
        metrics=["acc"]
    )
    
    early_stopping = EarlyStopping("val_loss", patience=PATIENCE)
    check_point = ModelCheckpoint(
        "./log/%s.Multi_LSTM_CNN_v1.word.{epoch:03d}.hdf5" % (datetime.now().strftime("%Y%m%d-%H%M%S")),  # word/char switch
        monitor="val_loss",
        save_best_only=True,
    )
    
    fit_res = model.fit(
        x=[train_x1, train_x2],
        y=train_y,
        batch_size=BATCH_SIZE,
        epochs=NUM_EPOCHES,
        validation_data=([dev_x1, dev_x2], dev_y),
        shuffle=True,
        callbacks=[early_stopping, check_point]
    )
    
    pred_last = model.predict([test_word1, test_word2], batch_size=BATCH_SIZE)  # word/char switch
    last_results.append(pd.DataFrame(pred_last, columns=["y_pre"]))
    
    print("load model %s" % (glob("./log/*.hdf5")[-1].replace("\\", "/"),))
    model.load_weights(glob("./log/*.hdf5")[-1].replace("\\", "/"))
    pred_best = model.predict([test_word1, test_word2], batch_size=BATCH_SIZE)  # word/char switch
    best_results.append(pd.DataFrame(pred_best, columns=["y_pre"]))

pd.DataFrame(pd.concat(last_results, axis=1).mean(axis=1), columns=["y_pre"]).to_csv(
    "./result/%s-Multi_LSTM_CNN_v1_word_last.csv" % (datetime.now().strftime("%Y%m%d-%H%M%S")),  # word/char switch
    index=False
)
pd.DataFrame(pd.concat(best_results, axis=1).mean(axis=1), columns=["y_pre"]).to_csv(
    "./result/%s-Multi_LSTM_CNN_v1_word_best.csv" % (datetime.now().strftime("%Y%m%d-%H%M%S")),  # word/char switch
    index=False
)

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
Train on 228946 samples, validate on 25440 samples
Epoch 1/30
228946/228946 [==============================] - 167s 729us/step - loss: 0.4562 - acc: 0.7857 - val_loss: 0.3437 - val_acc: 0.8555
Epoch 2/30
228946/228946 [==============================] - 162s 707us/step - loss: 0.3170 - acc: 0.8602 - val_loss: 0.2723 - val_acc: 0.8814
Epoch 3/30
228946/228946 [==============================] - 159s 694us/step - loss: 0.2703 - acc: 0.8825 - val_loss: 0.2346 - val_acc: 0.9001
Epoch 4/30
228946/228946 [==============================] - 164s 714us/step - loss: 0.2431 - acc: 0.8959 - val_loss: 0.2552 - val_acc: 0.8983
Epoch 5/30
228946/228946 [==============================] - 165s 722us/step - loss: 0.2257 - acc: 0.9041 - val_loss: 0.2271 - val_acc: 0.9105
Epoch 6/30
228946/228946 [==============================] - 170s 741us/step - loss: 0.2109 - acc: 0.9109 - val_loss: 0.2069 - val_acc: 0.9182
Epoch 7/30
228

Epoch 6/30
228948/228948 [==============================] - 168s 734us/step - loss: 0.2148 - acc: 0.9091 - val_loss: 0.2095 - val_acc: 0.9160
Epoch 7/30
228948/228948 [==============================] - 168s 734us/step - loss: 0.2049 - acc: 0.9141 - val_loss: 0.2010 - val_acc: 0.9183
Epoch 8/30
228948/228948 [==============================] - 159s 695us/step - loss: 0.1960 - acc: 0.9190 - val_loss: 0.2006 - val_acc: 0.9191
Epoch 9/30
228948/228948 [==============================] - 166s 726us/step - loss: 0.1896 - acc: 0.9207 - val_loss: 0.1904 - val_acc: 0.9238
Epoch 10/30
228948/228948 [==============================] - 167s 731us/step - loss: 0.1825 - acc: 0.9243 - val_loss: 0.2000 - val_acc: 0.9191
Epoch 11/30
228948/228948 [==============================] - 169s 738us/step - loss: 0.1774 - acc: 0.9267 - val_loss: 0.1864 - val_acc: 0.9257
Epoch 12/30
228948/228948 [==============================] - 167s 730us/step - loss: 0.1741 - acc: 0.9277 - val_loss: 0.2013 - val_acc: 0.9211
Epo

Epoch 13/30
228948/228948 [==============================] - 168s 733us/step - loss: 0.1684 - acc: 0.9298 - val_loss: 0.1880 - val_acc: 0.9278
Epoch 14/30
228948/228948 [==============================] - 167s 730us/step - loss: 0.1636 - acc: 0.9317 - val_loss: 0.1852 - val_acc: 0.9296
Epoch 15/30
228948/228948 [==============================] - 167s 730us/step - loss: 0.1615 - acc: 0.9335 - val_loss: 0.1829 - val_acc: 0.9298
Epoch 16/30
228948/228948 [==============================] - 168s 735us/step - loss: 0.1571 - acc: 0.9351 - val_loss: 0.1899 - val_acc: 0.9275
Epoch 17/30
228948/228948 [==============================] - 169s 740us/step - loss: 0.1553 - acc: 0.9359 - val_loss: 0.1886 - val_acc: 0.9302
Epoch 18/30
228948/228948 [==============================] - 167s 728us/step - loss: 0.1519 - acc: 0.9368 - val_loss: 0.1863 - val_acc: 0.9290
Epoch 19/30
228948/228948 [==============================] - 168s 733us/step - loss: 0.1487 - acc: 0.9383 - val_loss: 0.1809 - val_acc: 0.9329

Epoch 12/30
228948/228948 [==============================] - 159s 695us/step - loss: 0.1703 - acc: 0.9293 - val_loss: 0.1852 - val_acc: 0.9277
Epoch 13/30
228948/228948 [==============================] - 159s 692us/step - loss: 0.1662 - acc: 0.9314 - val_loss: 0.1881 - val_acc: 0.9251
Epoch 14/30
228948/228948 [==============================] - 161s 701us/step - loss: 0.1637 - acc: 0.9321 - val_loss: 0.1931 - val_acc: 0.9233
Epoch 15/30
228948/228948 [==============================] - 169s 739us/step - loss: 0.1591 - acc: 0.9342 - val_loss: 0.1828 - val_acc: 0.9273
Epoch 16/30
228948/228948 [==============================] - 168s 732us/step - loss: 0.1563 - acc: 0.9352 - val_loss: 0.1891 - val_acc: 0.9259
Epoch 17/30
228948/228948 [==============================] - 168s 735us/step - loss: 0.1542 - acc: 0.9365 - val_loss: 0.1928 - val_acc: 0.9283
Epoch 18/30
228948/228948 [==============================] - 168s 734us/step - loss: 0.1499 - acc: 0.9379 - val_loss: 0.1943 - val_acc: 0.9268